In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# pip install pycham

# 순전파
순전파 Function, Class 구현

In [ ]:
# 역전파에 대응하는 Varaiable 클래스 구현
# - data와 그에 대응하는 미분값(grad) 저장
class Variable:
  def __init__(self,data):
    self.data = data
    self.grad = None

In [ ]:
data = np.array(1.0)
x = Variable(data)

print(x.data)
print(x)
x.data

1.0


array(1.)

In [ ]:
class Fuction:
  def __call__(self, input):
    x = input.data
    y = x**2
    output = Variable(y)
    return output

In [ ]:
x = Variable(np.array(10))
f = Fuction()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [ ]:
class Function:
  def __call__(self,input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output

  def forward(self, in_data):
    raise NotImplementedError()

class Square(Function):
  def forward(self, x):
    return x **2

class Exp(Function):
  def forward(self, x):
    return np.exp(x)

In [ ]:
def numerical_diff(f,x,eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data-y0.data) / (2*eps)

f = Square()
x = Variable(np.array(2))
dy = numerical_diff(f,x)
print(dy)

4.000000000004


In [ ]:
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f,x)
print(dy)


3.2974426293330694


# 역전파 구현

순전파 구현한 함수에 역전파 함수 입력
backword

In [ ]:
# 기울기
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    # funcs = [self.creator]
    # while funcs:
    #   f = funcs.pop()
    #   x, y = f.input, f.output
    #   x.grad = f.backward(y.grad)

    #   if x.creator is not None:
    #     funcs.append(x.creator)
    f = self.creator
    if f is not None:
      x = f.input
      x.grad = f.backward(self.grad)
      x.backward()

class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input
    return output

  # 순전파
  def forward(self, x):
    raise NotImplementedError()

  # 역전파
  def backward(self, gy):
    raise NotImplementedError()


In [ ]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

class Exp(Function):
  def forward(self, x):
    return np.exp(x)

  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [ ]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x. grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# creator 을 이용한 역전파 도전

In [ ]:
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(b.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

AttributeError: 'NoneType' object has no attribute 'input'